# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
#конкатенация
foldr(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [6]:
def swap(func):
    def newF(x, y):
        return func(y, x)
    return newF

In [7]:
def foldl2(f, x0, lst):
    return foldr(swap(f), x0, lst[::-1])

In [8]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

In [9]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3, 4, 5])

'12345'

In [10]:
#сумма
foldl2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [11]:
def foldr2(f, x0, lst):
    return foldl(swap(f), x0, lst[::-1])

In [12]:
f = lambda x, y: x / y
foldr2(f, 1, [1, 2, 3])

1.5

In [13]:
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3, 4, 5])

'12345'

In [14]:
#сумма
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

6

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [15]:
def check_inv(a, b):
    xor = 0
    for j in b:
        xor ^= ord(j) 
    for i in range(len(a)-len(b)):
        xori = xor
        a1 = a[i:i+len(b)]
        for j in a1:
            xori ^= ord(j) 
        if xori == 0:
            return True
    return False

O(m*n)

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [16]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            yield from self.left
        if not self.left and not self.right:
            yield self.value
        if self.right:
            yield from self.right
        
    def __str__(self):
        return '{}, {}, {}'.format(
          str(self.value),
          str(self.left),
          str(self.right)) \
               .replace(', None, None', '') \
               .replace(', None', '')
    
    def __repr__(self):
        return '{}({}, {}, {})'.format(
          self.__class__.__name__,
          repr(self.value),
          repr(self.left),
          repr(self.right)) \
               .replace(', None, None)', ')') \
               .replace(', None)', ')')

    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]
repr(tree)

'Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))'

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [17]:
def calc(expr):
    return eval(expr)
calc('2 * (15 - 3 * 4) - 2') == 4

True

In [18]:
OPERATORS = {'+': (1, lambda x, y: x + y),
             '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y),
             '/': (2, lambda x, y: x / y)}



def parse(string):
    num = ''
    for s in string:
        if s in '0123456789':
            num += s
        elif num:
            yield float(num)
            num = ''
        if s in OPERATORS or s in "()":
            yield s
    if num:
        yield float(num)

def shunting_yard(pars):
    stack = []
    for symb in pars:
        if symb in OPERATORS:
            while stack and stack[-1] != "(" and OPERATORS[symb][0] <= OPERATORS[stack[-1]][0]:
                yield stack.pop()
            stack.append(symb)
        elif symb == ")":
            while stack:
                x = stack.pop()
                if x == "(":
                    break
                yield x
        elif symb == "(":
            stack.append(symb)
        else:
            yield symb
    while stack:
        yield stack.pop()

def calc(expr):
    x = parse(expr)
    polish = shunting_yard(x)
    stack = []
    for symb in polish:
        if symb in OPERATORS:
            a = stack.pop()
            b = stack.pop()
            stack.append(OPERATORS[symb][1](b, a))
        else:
            stack.append(symb)
    return stack[0]

calc('2 * (15 - 3 * 4) - 2') == 4

True